In [1]:
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\visha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
# Download sentence tokenizer
nltk.data.path.append('.')

In [4]:
import re
from nltk.tokenize import word_tokenize

path = "C:/Users/visha/OneDrive/Desktop/bts.txt"
with open(path, 'r', encoding='utf-8') as file:
    data: str = file.read()

data = re.sub(r'[,!?;-]', '.', data)
data = word_tokenize(data)


In [5]:
print(f"The Number of tokens = {len(data)} \n {data[:20]}")

The Number of tokens = 1686 
 ['BTS', '(', 'Korean', 'RR', ':', 'Bangtan', 'Sonyeondan', '.', 'lit', '.', 'Bulletproof', 'Boy', 'Scouts', ')', '.', 'also', 'known', 'as', 'the', 'Bangtan']


In [6]:
# Compute the frequency distribution of the words in the dataset (vocabulary)
fdist = nltk.FreqDist(word for word in data)
print("Size of vocabulary: ",len(fdist) )
print("Most frequent tokens: ",fdist.most_common(20) )

Size of vocabulary:  637
Most frequent tokens:  [('.', 151), ('the', 76), ('and', 44), ('in', 43), ('[', 42), (']', 42), ('of', 36), ('to', 32), ('``', 24), ("''", 24), ('their', 20), ('on', 19), ('BTS', 18), ('The', 18), ('Korean', 16), ('(', 15), (':', 15), (')', 15), ('a', 15), ('first', 14)]


In [7]:
# get_dict creates two dictionaries, converting words to indices and viceversa.
def get_dict(data: list[str]) -> tuple:
    words = sorted(list(set(data)))
    n = len(words)
    idx = 0
    word2Ind = {}
    Ind2word = {}
    for k in words:
        word2Ind[k] = idx
        Ind2word[idx] = k
        idx += 1
    return word2Ind, Ind2word


word2Ind, Ind2word = get_dict(data)
V = len(word2Ind)
print("Size of vocabulary: ", V)

Size of vocabulary:  637


In [12]:
print('bts' in word2Ind) 
print(list(word2Ind.keys())[:10])  


False
['&', "'", "''", "'Bulletproof", "'s", '(', ')', '.', '1.', '10']


In [13]:
def init_parameters(N: int, V: int, random_seed: int= 1) -> dict:
    np.random.seed(random_seed)
    parameters = {}

    parameters["W1"] = np.random.rand(N, V)
    parameters["b1"] = np.zeros(shape=(N, 1))

    parameters["W2"] = np.random.rand(V, N)
    parameters["b2"] = np.zeros(shape=(V, 1))

    return parameters

In [14]:
# Test he function example.
tmp_N = 4
tmp_V = 10
params = init_parameters(tmp_N,tmp_V)
assert params['W1'].shape == ((tmp_N,tmp_V))
assert params['W2'].shape == ((tmp_V,tmp_N))
print(f"W1 shape: {params['W1'].shape}")
print(f"W2 shape: {params['W2'].shape}")
print(f"b1 shape: {params['b1'].shape}")
print(f"b2 shape: {params['b2'].shape}")

W1 shape: (4, 10)
W2 shape: (10, 4)
b1 shape: (4, 1)
b2 shape: (10, 1)


In [15]:
 # desine the activation function that are will be used in our NN later.
from typing import List, Union

def softmax(z: Union[float, List[float]]) -> Union[float, List[float]]:
    y = np.exp(z) / np.sum(np.exp(z), 0, keepdims=True)

    return y

def sigmoid(z: Union[float, List[float]]) -> Union[float, List[float]]:
    return 1.0 / (1.0 + np.exp(-z))

def relu(z: Union[float, List[float]]) -> Union[float, List[float]]:
    return np.maximum(z, 0)

In [16]:
tmp = np.array([[1,2,3],
                [1,1,1]
               ])
tmp_sm = softmax(tmp)
display(tmp_sm)

array([[0.5       , 0.73105858, 0.88079708],
       [0.5       , 0.26894142, 0.11920292]])

In [17]:
def forward_propagation(x: np.ndarray, params: dict) -> tuple:
    z1 = np.dot(params['W1'], x) + params['b1']
    h = relu(z1)

    z2 = np.dot(params['W2'], h) + params['b2']
    y_hat = z2

    return y_hat, h

In [18]:
# test the function
tmp_x = np.array([[0,1,0,0,0,0,0,0,0,0]]).T
tmp_z, tmp_h = forward_propagation(tmp_x, params)
print("call forward_prop")
print()
# Look at output
print(f"z has shape {tmp_z.shape}")
print("z has values:")
print(tmp_z)

call forward_prop

z has shape (10, 1)
z has values:
[[1.82887324]
 [1.38466287]
 [0.60100484]
 [0.83284144]
 [1.37936774]
 [1.20420827]
 [1.26273995]
 [2.0149547 ]
 [1.10825153]
 [1.93910889]]


In [19]:
def cross_entropy_loss(y_hat: np.ndarray, y: np.ndarray,
                       batch_size: int) -> Union[float, List[float]]:
    logprobs = np.multiply(np.log(y_hat), y)
    cost = -1/batch_size * np.sum(logprobs)
    cost = np.squeeze(cost)
    return cost

In [20]:
def backward_propagation(x: np.ndarray, y: np.ndarray, y_hat: np.ndarray, h: np.ndarray,
                         params: dict, batch_size: int) -> dict:
    grads_params = {}

    l1 = np.dot(params['W2'].T ,(y_hat - y))

    grads_params['W1'] = np.dot(l1, x.T) / batch_size
    grads_params['W2'] = np.dot((y_hat - y), h.T) / batch_size

    grads_params['b1'] = np.sum(l1, axis=1, keepdims= True) / batch_size
    grads_params['b2'] = np.sum((y_hat - y), axis= 1, keepdims= True) / batch_size

    return grads_params

In [21]:
from collections import defaultdict

def get_idx(words, word2Ind):
    idx = []
    for word in words:
        idx = idx + [word2Ind[word]]
    return idx

def pack_idx_with_frequency(context_words, word2Ind):
    freq_dict = defaultdict(int)
    for word in context_words:
        freq_dict[word] += 1
    idxs = get_idx(context_words, word2Ind)
    packed = []
    for i in range(len(idxs)):
        idx = idxs[i]
        freq = freq_dict[context_words[i]]
        packed.append((idx, freq))
    return packed

def get_vectors(data, word2Ind, V, C):
    i = C
    while True:
        y = np.zeros(V)
        x = np.zeros(V)
        center_word = data[i]
        y[word2Ind[center_word]] = 1
        context_words = data[(i - C) : i] + data[(i + 1) : (i + C + 1)]
        num_ctx_words = len(context_words)
        for idx, freq in pack_idx_with_frequency(context_words, word2Ind):
            x[idx] = freq / num_ctx_words
        yield x, y
        i += 1
        if i >= len(data) - C:
            print("i is being set to", C)
            i = C

def get_batches(data, word2Ind, V, C, batch_size):
    batch_x = []
    batch_y = []
    for x, y in get_vectors(data, word2Ind, V, C):
        if len(batch_x) < batch_size:
            batch_x.append(x)
            batch_y.append(y)
        else:
            yield np.array(batch_x).T, np.array(batch_y).T
            batch_x = []
            batch_y = []

In [22]:
def gradient_descent(data: list[str], word2Ind: dict, N: int,
                     V: int, epochs: int, alpha: float= 0.03,
                     random_seed: int= 282) -> dict:

    param = init_parameters(N,V, random_seed=random_seed)

    batch_size = 128
    epoch = 0
    C = 2

    for x, y in get_batches(data, word2Ind, V, C, batch_size):

        z, h = forward_propagation(x, param)
        yhat = softmax(z)

        cost = cross_entropy_loss(yhat, y, batch_size)
        if ( (epoch+1) % 10 == 0):
            print(f"iters: {epoch + 1} cost: {cost:.6f}")


        grads = backward_propagation(x,y, yhat, h, param, batch_size)


        param['W1'] = param['W1'] - alpha * grads['W1']
        param['W2'] = param['W2'] - alpha * grads['W2']
        param['b1'] = param['b1'] - alpha * grads['b1']
        param['b2'] = param['b2'] - alpha * grads['b2']

        epoch +=1
        if epoch == epochs:
            break
        if epoch % 100 == 0:
            alpha *= 0.66

    return param

In [23]:
# test the function
C = 2
N = 50
word2Ind, Ind2word = get_dict(data)
V = len(word2Ind)
num_iters = 150
print("Call gradient_descent")
params = gradient_descent(data, word2Ind, N, V, num_iters)

Call gradient_descent
iters: 10 cost: 6.827731
i is being set to 2
iters: 20 cost: 6.802057
i is being set to 2
iters: 30 cost: 6.639857
i is being set to 2
iters: 40 cost: 6.598190
iters: 50 cost: 6.633337
i is being set to 2
iters: 60 cost: 6.499403
i is being set to 2
iters: 70 cost: 6.407509
i is being set to 2
iters: 80 cost: 6.284237
iters: 90 cost: 6.338937
i is being set to 2
iters: 100 cost: 6.094671
i is being set to 2
iters: 110 cost: 6.238966
i is being set to 2
iters: 120 cost: 6.206588
iters: 130 cost: 6.209357
i is being set to 2
iters: 140 cost: 5.965101
i is being set to 2
iters: 150 cost: 6.091380
